In [1]:
import sys
import os
import json
from shapely.geometry import shape
import numpy as np
import xarray as xr
from pystac_client import Client as StacClient
from dask.distributed import Client, LocalCluster
import stackstac
from typing import Dict
import matplotlib.pyplot as plt
import rioxarray

In [2]:

cluster = LocalCluster(
    n_workers=3,             # safe for ~12–15 GB RAM
    threads_per_worker=1,    # recommended for raster ops
    memory_limit="4GB"       # 3 workers × 4GB = 12GB total
)
client = Client(cluster)
cluster    # This will display cluster dashboard info


/opt/conda/envs/geog313-final-project/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40495 instead
  warnings.warn(


LocalCluster(1365559b, 'tcp://127.0.0.1:40921', workers=3, threads=3, memory=11.18 GiB)

In [5]:
# Add the folder containing WQI_utils.py
sys.path.append(os.path.abspath("../utils"))

## 2 Import the main functions
from env_utils import environmental_variables
from plots_utils import plot_time_series, plot_seasonal_cycle
from wrapper_plots_utils import workflow_analysis

In [8]:
bbox = [-82.8, 27.5, -82.2, 28.2]  # Tampa Bay

env_data = environmental_variables(
    bbox=bbox,
    start_year=2019,
    end_year=2024,
    variables=["sst", "precipitation"]
)
# Fetch items per year
items = []
for year in range(start_year, end_year + 1):
    search = client.search(
        collections=[collection],
        bbox=bbox,
        datetime=f"{year}-01-01/{year}-12-31",
        max_items=100
    )
    items.extend(list(search.get_items()))

# Debug: check items
print(f"Number of items found for {var}: {len(items)}")
for i, item in enumerate(items[:5]):
    print(f"Item {i}: geometry={item.geometry}, assets={list(item.assets.keys())}")


/opt/conda/envs/geog313-final-project/lib/python3.11/site-packages/pystac_client/item_search.py:835: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(


AssertionError: out_bounds=None

In [5]:
bbox = [-82.8, 27.5, -82.2, 28.2]  # Tampa Bay
env_data = environmental_variables(
    bbox=bbox,
    start_year=2019,
    end_year=2024,
    variables=["sst", "precipitation"]
)

# Access xarray time series
sst_series = env_data["sst"]
precip_series = env_data["precipitation"]

# Quick plot
#sst_series.plot.line(title="SST (Tampa Bay)")
#precip_series.plot.line(title="Precipitation (Tampa Bay)")


/opt/conda/envs/geog313-final-project/lib/python3.11/site-packages/pystac_client/item_search.py:835: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(


AssertionError: out_bounds=None

In [10]:
from pystac_client import Client

# Test MODIS SST
stac_url = "https://cmr.earthdata.nasa.gov/stac/LANCEMODIS"
client = Client.open(stac_url)
items = client.search(
    collections=["MODIS_Aqua_L3SM_SST"],
    bbox=bbox,
    datetime="2022-01-01/2022-12-31",
    max_items=10
).get_all_items()

print("Number of SST items:", len(items))


/opt/conda/envs/geog313-final-project/lib/python3.11/site-packages/pystac_client/item_search.py:850: FutureWarning: get_all_items() is deprecated, use item_collection() instead.
  warnings.warn(


Number of SST items: 0
